In [1]:
pip install transformers torch pandas scikit-learn


In [11]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
# Load dataset
data = pd.read_csv('Binary_Class_Balanced_Dataset_4Lac_60_Features.csv')

In [6]:
# Load dataset
data = pd.read_csv('Binary_Class_Balanced_Dataset_4Lac_60_Features.csv')

# Print the column names
print(data.columns)


Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'SYN Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'Down/Up Ratio', '

In [8]:
# Menggunakan semua fitur kecuali 'Label' sebagai input
X = data.drop(columns=['Label']).values  # Menghapus kolom 'Label'
y = data['Label'].values  # Mengambil kolom 'Label' sebagai target

In [9]:
# Split dataset menjadi training dan validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Standardisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [13]:
# Dataset class
class TabularDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return {'input': torch.tensor(feature, dtype=torch.float32), 'label': torch.tensor(label)}


In [14]:
# Create DataLoader
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [15]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Layer pertama
        self.fc2 = nn.Linear(128, 64)           # Layer kedua
        self.fc3 = nn.Linear(64, num_classes)   # Layer output
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [16]:
# Initialize model, loss function, and optimizer
input_size = X_train.shape[1]  # Jumlah fitur
num_classes = len(set(y))       # Jumlah kelas
model = MLP(input_size, num_classes)

criterion = nn.CrossEntropyLoss()  # Loss function untuk klasifikasi
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):  # number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = batch['input'].to(device)
        labels = batch['label'].to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.1208152249455452
Epoch 2, Loss: 0.12760938704013824
Epoch 3, Loss: 0.06483794748783112
Epoch 4, Loss: 5.975149633741239e-06
Epoch 5, Loss: 0.058578409254550934
Epoch 6, Loss: 2.0116412997595035e-06
Epoch 7, Loss: 0.03256457298994064
Epoch 8, Loss: 0.00020960108668077737
Epoch 9, Loss: 0.08911853283643723
Epoch 10, Loss: 0.05720480531454086


In [18]:
# Save the model
torch.save(model.state_dict(), 'mlp_model.pth')

# **Transformer Model Bert**

**1. Pendahuluan**

Proyek ini bertujuan untuk membangun model klasifikasi menggunakan model transformer BERT (Bidirectional Encoder Representations from Transformers). Dataset yang digunakan adalah dataset yang berisi fitur-fitur numerik yang akan diolah menjadi representasi teks. Model ini bertujuan untuk memprediksi label berdasarkan input yang diberikan.

**2. Persiapan Data**

**2.1 Memuat Dataset**

Dataset berhasil dimuat menggunakan Pandas, dan semua data yang diperlukan tersedia untuk proses selanjutnya.

**2.2 Pengolahan Data**

 Pengolahan data dilakukan untuk menyiapkan format input yang diperlukan oleh BERT. Meskipun fitur numerik diubah menjadi teks untuk keperluan ini, penting untuk memastikan bahwa pendekatan ini relevan dengan konteks aplikasi yang diinginkan.

 **2.3 Pemisahan Fitur dan Label**

 Fitur dan label berhasil dipisahkan, sehingga siap untuk digunakan dalam pelatihan model.


**2.4 Encoding Label**

Label dikodekan ke dalam format numerik yang memungkinkan model untuk memahami kategori yang harus diprediksi, sehingga mempersiapkan data untuk proses pelatihan.

**2.5 Pembagian Dataset**

Dataset berhasil dibagi menjadi data latih dan validasi, yang penting untuk menilai kinerja model pada data yang tidak terlihat selama pelatihan.

**3. Tokenisasi**

Tokenisasi menggunakan tokenizer BERT berhasil dilakukan, yang mengubah teks menjadi representasi yang dapat dipahami oleh model BERT. Ini adalah langkah kunci dalam mempersiapkan data untuk proses pelatihan.

**4. Membuat Dataset Kustom**

Kelas dataset kustom berhasil dibuat, memungkinkan pengelolaan data input dan label dengan efisien. Ini memfasilitasi proses pembelajaran model dengan cara yang lebih terstruktur.

**5. DataLoader**

DataLoader disiapkan untuk mempermudah pengambilan batch data selama pelatihan, meningkatkan efisiensi dan kecepatan proses pelatihan model.

**6. Membangun Model**

Model BERT berhasil diinisialisasi dengan parameter yang sesuai untuk tugas klasifikasi, menyediakan fondasi yang kuat untuk pembelajaran.

**7. Optimizer**

Optimizer AdamW dipilih untuk meningkatkan efisiensi proses pembelajaran, yang diharapkan dapat mempercepat konvergensi model.

**8. Pelatihan Model**

Model dilatih dengan sukses melalui beberapa epoch, dan proses pelatihan menghasilkan penurunan nilai loss, menunjukkan bahwa model belajar dari data. Pemantauan loss selama pelatihan membantu dalam mengidentifikasi kemajuan model.

**9. Menyimpan Model**

Model yang telah dilatih disimpan dengan baik untuk digunakan di masa mendatang. Ini memungkinkan pemanfaatan model tanpa perlu melatih ulang, menghemat waktu dan sumber daya.

**10. Hasil dan Analisis**

Hasil pelatihan menunjukkan bahwa model dapat belajar dengan baik, dengan penurunan nilai loss dari epoch ke epoch. Namun, evaluasi lebih lanjut diperlukan untuk memahami seberapa baik model ini bekerja pada data baru dan untuk mengeksplorasi metrik evaluasi yang lebih mendalam.

**10.1 Evaluasi Model**

Evaluasi model sangat penting untuk menilai kinerja dan akurasi model. Metrik seperti akurasi, presisi, dan recall dapat memberikan wawasan tambahan tentang bagaimana model beroperasi dalam konteks klasifikasi.

**11. Kesimpulan**

Model klasifikasi berbasis BERT berhasil dibangun dan menunjukkan potensi yang baik dalam memprediksi label berdasarkan input teks. Meskipun penggunaan data numerik dalam format teks berfungsi untuk tujuan ini, perlu dicatat bahwa penggunaan data yang lebih relevan dan representatif akan sangat membantu dalam meningkatkan performa model. Tuning hyperparameter dan eksplorasi lebih lanjut terhadap data dapat dilakukan untuk mencapai hasil yang lebih baik.

